<a href="https://colab.research.google.com/github/kifoo/Sieci_neuronowe/blob/main/SN_projekt1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kod implementacji płystkiej sieci neuronowej

In [ ]:
!pip install ucimlrepo
# !pip import pandas
# !pip install -U scikit-learn
# !pip install keras
!pip install tensorflow

Importing required libraries

In [11]:
import numpy as np
import pandas as pd
import os
import csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow import keras
from keras import models, layers, callbacks
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from ucimlrepo import fetch_ucirepo

ImportError: DLL load failed while importing _pywrap_tf2: Procedura inicjowania biblioteki dołączanej dynamicznie (DLL) nie powiodła się.

Function fetching dataset from https://archive.ics.uci.edu/dataset/602/dry+bean+dataset

In [ ]:
def data_fetch():
    # fetch dataset
    dry_bean_dataset = fetch_ucirepo(id=602)

    # data (as pandas dataframes)
    X = dry_bean_dataset.data.features
    Y = dry_bean_dataset.data.targets

    label_encoder = LabelEncoder()
    Y = label_encoder.fit_transform(Y)
    return X, Y

: 

Function defining the neural network model

In [ ]:
# Define the neural network model
def create_model(input_shape, output_shape, nuerons_output, hidden_layers):
    model = Sequential()
    model.add(Dense(nuerons_output, activation='relu', input_shape=(input_shape,)))
    for i in range(hidden_layers):
        model.add(Dense(nuerons_output, activation='relu'))
    model.add(Dense(output_shape, activation='softmax'))
    return model

: 

Creating the neural network model, training it and testing it accuracy

In [ ]:
def neural_network(opt_function, loss_function, nuerons_output, hidden_layers, epochs):
    # This function return the dataframe
    X, Y = data_fetch()

    # Split the dataset into a training set and a testing set
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.5)

    # Normalize the inputs
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # print("========== Train ==========")
    # print(y_train.shape[0])
    # print("========== Test ==========")
    # print(X_train.shape[1])

    # Create the neural network model
    model = create_model(input_shape=X_train.shape[1], output_shape=7, nuerons_output=nuerons_output, hidden_layers=hidden_layers)

    # Compile the model
    model.compile(optimizer=opt_function, loss=loss_function, metrics='accuracy')

    # Train the model
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs)

    # Test the model
    _, accuracy = model.evaluate(X_test, y_test)
    print(f"Test Accuracy: {accuracy}")
    return accuracy

: 

Loss function exapmles:

**'mean_squared_error' or 'mse':** Measures the average squared difference between
the true and predicted labels. It is useful for regression problems.

**'binary_crossentropy':** Applies a binary cross-entropy between the true and predicted labels. It is useful for binary classification problems.

**'categorical_crossentropy':** Applies a categorical cross-entropy between the true and predicted labels. It is useful for multi-class classification problems.

**'hinge':** Defines a hinge loss function that measures the difference between the predicted labels and true labels.

**'logcosh':** Defines a logarithm of the hyperbolic cosine loss function.

**'sparse_categorical_crossentropy':** Similar to 'categorical_crossentropy', but it can accept labels with integer values instead of one-hot encoded vectors.

**'kl_divergence':** Calculates the Kullback-Leibler divergence, which quantifies the difference between two probability distributions.

**'poisson':** Calculates the Poisson loss between the true and predicted labels.

In [ ]:
def write_data(neuron_outputs,hidden_layers,epochs,acc):
    csv_file_path ="content/data.csv"

    with open(csv_file_path, 'a', newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow([neuron_outputs, hidden_layers, epochs, acc])
        print("ok\n")


def main():
    loss = ['mean_squared_error', 'binary_crossentropy', 'categorical_crossentropy', 'hinge', 'logcosh', 'sparse_categorical_crossentropy', 'kl_divergence']
    neuron_outputs = [2, 4, 8, 12, 16]
    hidden_layers = [0, 1, 2]  # góra-cy, ale liczymy od 0 jak prawdziwi informatycy XDD
    epochs = [5, 10, 25, 50]

    # Ej bo tu wychodzi jakieś 60 danych XDD Dokładnie to 63, nie wiem czemu XDD
    min_acc = 1
    min_data = np.array([[0], [0], [0], [0.0]])
    max_acc = 0
    max_data = np.array([[0], [0], [0], [0.0]])
    write_data("Neuron Output","Hidden Layers","Epochs","Accuracy")

    for a in range(len(neuron_outputs)):
          for b in range(len(hidden_layers)):
              for c in range(len(epochs)):
                  acc = neural_network(opt_function='Adam', loss_function=loss[5], nuerons_output=neuron_outputs[a], hidden_layers=hidden_layers[b], epochs=epochs[c])
                  if( acc > max_acc):
                      max_acc = acc
                      max_data = [[neuron_outputs[a]], [hidden_layers[b]], [epochs[c]], [acc]]
                  if( acc < min_acc):
                      min_acc = acc
                      min_data = [[neuron_outputs[a]], [hidden_layers[b]], [epochs[c]], [acc]]
                  write_data(neuron_outputs[a],hidden_layers[b],epochs[c],acc)

    print(f"Min accuracy: {min_data} \nMax accuracy: {max_data}")

main()

: 

Min accuracy: [[2], [2], [50], [0.2634440064430237]]

Max accuracy: [[16], [1], [50], [0.9315310120582581]]